1、本节目标
---
抓取猫眼电影TOP100的电影名称、时间、评分、图片等信息，提取的站点URL为http://maoyan.com/board/4 ，提取的结果会以文件形式保存下来
---
2、准备工作
---
安装requests库
---
3、抓取分析
---
观察点击第二页之后URL的变化（每次offset += 10），offset代表偏移量，如果偏移量为n，则显示的电影序号就是n+1到n+10，每页显示10个。要想获取TOP100，，只需要分开请求10次，十次的offset分别设置为0、10、20、。。。90即可，获取不同页面后，通过正则表达式提取出相关信息即可。
---
4、抓取首页
---

In [2]:
import requests

def get_one_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)' +
        'AppleWebKit/537.36 (KHTML, like Gecko)' +
        'Chrome/72.0.3626.121 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    return None

def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    print(html)

main()

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 9]><!--><html><!--<![endif]-->
<head>
  <title>TOP100榜 - 猫眼电影 - 一网打尽好电影</title>
  
  <link rel="dns-prefetch" href="//p0.meituan.net"  />
  <link rel="dns-prefetch" href="//p1.meituan.net"  />
  <link rel="dns-prefetch" href="//ms0.meituan.net" />
  <link rel="dns-prefetch" href="//s0.meituan.net" />
  <link rel="dns-prefetch" href="//ms1.meituan.net" />
  <link rel="dns-prefetch" href="//analytics.meituan.com" />
  <link rel="dns-prefetch" href="//report.meituan.com" />
  <link rel="dns-prefetch" href="//frep.meituan.com" />

  
  <meta charset="utf-8">
  <meta name="keywords" content="猫眼电影,电影排行榜,热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100">
  <meta name="description" content="猫眼电影热门榜单,包括热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100,多维度为用户进行选片决策">
  <meta http-equiv="cleartype" content="yes" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="renderer" content="webki

以上就是获取到的首页源代码。然后需要解析页面，提取想要的信息。
---
5、正则提取
---
观察到一部电影对应的源代码是一个dd节点，我们用正则表达式来提取这里面的一些电影信息。首先，需要提取它的排名信息。而他的排名信息是在class为board-index的i节点内，这里利用非贪婪匹配来提取i节点内的信息，正则表达式写为： <dd>.*?board-index.*?>(.*?)</i>

随后需要提取电影的图片。可以看到，后面有a节点，其内部有两个img节点。经过检查后发现，第二个img节点的data-src属性是图片的链接。这里提取第二个img节点的data-src属性，正则表达式可以改写如下： <dd>.*?borad-index.*?>(.*?)</i>.*?data-src="(.*?)"

再往后，需要提取电影的名称，它在后面的p节点内，class为name。所以，可以用name做一个标志位，然后进一步提取到其内a节点的正文内容，此时正则表达式改写如下：
<dd>.*?borad-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>

再提取主演、发布时间、评分等内容时，都是同样的原理。最后，正则表达式写为：
<dd>.*?borad-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>

这样一个正则表达式可以匹配一个电影的结果，里面匹配了7个信息。接下来，调用findall()方法提取出所有的内容。

In [11]:
import re
def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    print(items)
parse_one_page(get_one_page('http://maoyan.com/board/4'))

[('1', 'https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', '霸王别姬', '\n                主演：张国荣,张丰毅,巩俐\n        ', '上映时间：1993-01-01', '9.', '5'), ('2', 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', '肖申克的救赎', '\n                主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿\n        ', '上映时间：1994-10-14(美国)', '9.', '5'), ('3', 'https://p0.meituan.net/movie/289f98ceaa8a0ae737d3dc01cd05ab052213631.jpg@160w_220h_1e_1c', '罗马假日', '\n                主演：格利高里·派克,奥黛丽·赫本,埃迪·艾伯特\n        ', '上映时间：1953-09-02(美国)', '9.', '1'), ('4', 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@160w_220h_1e_1c', '这个杀手不太冷', '\n                主演：让·雷诺,加里·奥德曼,娜塔莉·波特曼\n        ', '上映时间：1994-09-14(法国)', '9.', '5'), ('5', 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@160w_220h_1e_1c', '泰坦尼克号', '\n                主演：莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩\n        ', '上映时间：1998-04-03', '9.', '5'), ('6', 'https://p0.meituan.net/mov

观察上面的输出，数据比较杂乱，将匹配结果进行处理，遍历提取结果并生成字典，此时方法改写如下：

In [16]:
def parse_one_page_improved(html):
    pattern = re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield{
            'index': item[0],
            'image': item[1],
            'title': item[2].strip(),
            'actor': item[3].strip()[3:] if len(item[3]) > 3 else '',
            'time' : item[4].strip()[5:] if len(item[4]) > 5 else '',
            'score': item[5].strip() + item[6].strip()
        }
parse_one_page(get_one_page('http://maoyan.com/board/4'))


[('1', 'https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', '霸王别姬', '\n                主演：张国荣,张丰毅,巩俐\n        ', '上映时间：1993-01-01', '9.', '5'), ('2', 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', '肖申克的救赎', '\n                主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿\n        ', '上映时间：1994-10-14(美国)', '9.', '5'), ('3', 'https://p0.meituan.net/movie/289f98ceaa8a0ae737d3dc01cd05ab052213631.jpg@160w_220h_1e_1c', '罗马假日', '\n                主演：格利高里·派克,奥黛丽·赫本,埃迪·艾伯特\n        ', '上映时间：1953-09-02(美国)', '9.', '1'), ('4', 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@160w_220h_1e_1c', '这个杀手不太冷', '\n                主演：让·雷诺,加里·奥德曼,娜塔莉·波特曼\n        ', '上映时间：1994-09-14(法国)', '9.', '5'), ('5', 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@160w_220h_1e_1c', '泰坦尼克号', '\n                主演：莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩\n        ', '上映时间：1998-04-03', '9.', '5'), ('6', 'https://p0.meituan.net/mov

---
6、写入文件
---
随后，我们将提取的结果写入文件，这里直接写入到一个文本文件中。这里通过JSON库的dumps()方法实现字典的序列化，并指定ensure_ascii参数为False，这样可以保证输出结果是中文形式而不是Unicode编码。
---

In [ ]:
def write_to_file(content):
    with open('result.txt', 'a', encoding='utf-8') as f:
        print(type(json.dumps(content)))
        f.write(json.dumps(content, ensure_ascii=False)+'\n')

通过调用write_to_file()方法即可实现将字典写入到文本文件的过程，此处的content参数就是一部电影的提取结果，是一个字典。

---
7、整合代码
---
最后，实现main()方法来调用前面实现的方法，将单页的电影结果写入到文件。相关代码如下：

In [ ]:
def main():
    url = 'http://maoyan.com/board/4'
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)

---
8、分页爬取
---
遍历，传入offset参数
---

In [ ]:
if __name__ == '__main__':
    for i in range(10):
        main(offset=i * 10)
        
# 修改main() 方法修改一下，接收一个offset值作为偏移量，然后构造URL进行爬取。
def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)

整体代码如下：

In [3]:
import json 
import requests
from requests.exceptions import RequestException
import re
import time 

def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)' +
            'AppleWebKit/537.36 (KHTML, like Gecko)' +
            'Chrome/72.0.3626.121 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield{
            'index': item[0],
            'image': item[1],
            'title': item[2],
            'actor': item[3].strip()[3:],
            'time' : item[4].strip()[5:],
            'score': item[5] + item[6]
        }

def write_to_file(content):
    with open('./result.txt', 'a', encoding='utf-8') as f: 
        f.write(json.dumps(content, ensure_ascii=False) + '\n')

def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)

if __name__ == '__main__':
    for i in range(10):
        main(offset=i*10)
        time.sleep(1)




{'index': '1', 'image': 'https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', 'title': '霸王别姬', 'actor': '张国荣,张丰毅,巩俐', 'time': '1993-01-01', 'score': '9.5'}
{'index': '2', 'image': 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', 'title': '肖申克的救赎', 'actor': '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', 'time': '1994-10-14(美国)', 'score': '9.5'}
{'index': '3', 'image': 'https://p0.meituan.net/movie/289f98ceaa8a0ae737d3dc01cd05ab052213631.jpg@160w_220h_1e_1c', 'title': '罗马假日', 'actor': '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', 'time': '1953-09-02(美国)', 'score': '9.1'}
{'index': '4', 'image': 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@160w_220h_1e_1c', 'title': '这个杀手不太冷', 'actor': '让·雷诺,加里·奥德曼,娜塔莉·波特曼', 'time': '1994-09-14(法国)', 'score': '9.5'}
{'index': '5', 'image': 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@160w_220h_1e_1c', 'title': '泰坦尼克号', 'actor': '莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩', 'time': '199

{'index': '41', 'image': 'https://p0.meituan.net/movie/a08f65e6cb50fab32df5da69ff116f593095363.jpg@160w_220h_1e_1c', 'title': '幽灵公主', 'actor': '松田洋治,石田百合子,田中裕子', 'time': '1998-05-01', 'score': '8.9'}
{'index': '42', 'image': 'https://p0.meituan.net/movie/4f9638ba234c3fb673f23a09968db875371576.jpg@160w_220h_1e_1c', 'title': '风之谷', 'actor': '岛本须美,永井一郎,坂本千夏', 'time': '1992', 'score': '8.9'}
{'index': '43', 'image': 'https://p1.meituan.net/movie/06ec3c1c647942b1e40bca84036014e9490863.jpg@160w_220h_1e_1c', 'title': 'V字仇杀队', 'actor': '娜塔莉·波特曼,雨果·维文,斯蒂芬·瑞', 'time': '2006-03-17(美国)', 'score': '8.8'}
{'index': '44', 'image': 'https://p0.meituan.net/movie/df15efd261060d3094a73ef679888d4f238149.jpg@160w_220h_1e_1c', 'title': '十二怒汉', 'actor': '亨利·方达,李·科布,马丁·鲍尔萨姆', 'time': '1957-04-13(美国)', 'score': '9.1'}
{'index': '45', 'image': 'https://p1.meituan.net/movie/5896de3c1474277730e321c9b1db04a9205644.jpg@160w_220h_1e_1c', 'title': '当幸福来敲门', 'actor': '威尔·史密斯,贾登·史密斯,坦迪·牛顿', 'time': '2008-01-17', 'score

{'index': '81', 'image': 'https://p1.meituan.net/movie/d5970e36c8868a4b746c80f3b3f8a404174615.jpg@160w_220h_1e_1c', 'title': '穿条纹睡衣的男孩', 'actor': '阿沙·巴特菲尔德,维拉·法梅加,大卫·休里斯', 'time': '2008-09-12(英国)', 'score': '9.0'}
{'index': '82', 'image': 'https://p0.meituan.net/movie/3985eaf3858bea0f2a3d966bf7ee2103178217.jpg@160w_220h_1e_1c', 'title': '窃听风暴', 'actor': '乌尔里希·穆埃,塞巴斯蒂安·科赫,马蒂娜·格德克', 'time': '2006-03-23(德国)', 'score': '9.0'}
{'index': '83', 'image': 'https://p0.meituan.net/movie/4abc8c932cfacfc0089e2883765d02d1295222.jpg@160w_220h_1e_1c', 'title': '时空恋旅人', 'actor': '瑞秋·麦克亚当斯,多姆纳尔·格里森,比尔·奈伊', 'time': '2013-09-04(英国)', 'score': '8.9'}
{'index': '84', 'image': 'https://p1.meituan.net/movie/6a6e74b2c289f9fa4433dd2dc04a7741331638.jpg@160w_220h_1e_1c', 'title': '7号房的礼物', 'actor': '柳承龙,郑镇荣,朴信惠', 'time': '2013-01-23(韩国)', 'score': '8.9'}
{'index': '85', 'image': 'https://p0.meituan.net/movie/b5ff0216e689b3fcc065590c48cd5105255305.jpg@160w_220h_1e_1c', 'title': '恐怖直播', 'actor': '河正宇,李璟荣,李大为', 'tim